In [1]:
import pandas as pd; import numpy as np; import janitor as jn

In [2]:
url = "https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/flights.csv"
flights = pd.read_csv(url, sep = '\t', names=['orig','dest','orig_time', 'dest_time'], parse_dates  = ['orig_time', 'dest_time'])
flights = flights.factorize_columns(['orig','dest']).iloc[:, 2:]
flights.columns = [ent.split("_")[0] if ent.endswith('enc') else ent for ent in flights]
flights.columns = ['takeoff','landing','orig','dest']
flights = flights.assign(start=flights.landing+pd.Timedelta(minutes=45), end=flights.landing+pd.Timedelta(hours=3))
flights.head()

,takeoff,landing,orig,dest,start,end
0,2021-11-27 07:15:00,2021-11-27 08:55:00,0,0,2021-11-27 09:40:00,2021-11-27 11:55:00
1,2021-11-27 20:05:00,2021-11-28 00:50:00,1,1,2021-11-28 01:35:00,2021-11-28 03:50:00
2,2021-11-27 21:00:00,2021-11-27 21:35:00,2,2,2021-11-27 22:20:00,2021-11-28 00:35:00
3,2021-11-27 21:15:00,2021-11-27 22:25:00,0,2,2021-11-27 23:10:00,2021-11-28 01:25:00
4,2021-11-26 11:40:00,2021-11-26 14:45:00,3,3,2021-11-26 15:30:00,2021-11-26 17:45:00


In [3]:
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
    ('start', 'takeoff', '<='), 
     ('orig','orig','!='), 
     ('dest', 'orig', '=='), 
     df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=True)
)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


[ 42261  42262     48 ... 103962 107115 125899]


(array([     0,   5452,   7643, ..., 115186,  36075, 101440]),
 array([  5452,   7643,   8008, ..., 116033,  36215, 102364]))

In [4]:
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 'value_1': [2,5,7,1,3,4]})
df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 'value_2A': [0,3,7,12,0,2,3,1], 'value_2B': [1,9,5,15,1,6,4,3]})


In [5]:
df1.conditional_join(df2, ('id','id','=='), ('value_1', 'value_2A','>'), ('value_1','value_2B','<'), use_numba=True)

[0 1 2 3 4 5 6 7]


(array([0, 0, 0, 4, 4, 7]), array([4, 4, 4, 7, 7, 8]))

In [6]:
df1.conditional_join(df2, ('id','id','=='), ('value_1', 'value_2A','>'), ('value_1','value_2B','<'), use_numba=False)

(array([1, 2, 4]), array([1, 1, 5]))

In [7]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [8]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,9
2,1,7,5
3,1,12,15
4,2,0,1
5,2,2,6
6,2,3,4
7,3,1,3
